# Model the drizzled JWST image.

Perform this notebook requires standard astroconda libraries and the publicly available packages on github.

- lenstronomy (https://github.com/sibirrer/lenstronomy) (pip install lenstronomy)
- fastell4py (only when using the SPEMD model) (https://github.com/sibirrer/fastell4py), based on the original fastell fortran code (by Barkana)

This notebook has been tested with on Python 3.6.10, lenstronomy 1.3.0.

### Note that this notebook models the drizzled simulated image.


In [1]:
# import the necessary python modules

import numpy as np
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import glob
#from gen_fit_id import gen_fit_id
from photutils import make_source_mask
import os
import re

import sys
sys.path.insert(0,'../share_tools/')
from fit_qso import fit_qso
from transfer_to_result import transfer_to_result
from mask_objects import detect_obj
from flux_profile import profiles_compare, flux_profile
from matplotlib.colors import LogNorm
import copy
import time
import pickle

In [2]:
#Setting the fitting condition:
deep_seed = True  #Set as True to put more seed and steps to fit.
pltshow = 1 #Note that setting plt.ion() in line27, the plot won't show anymore if running in terminal.
pix_scale = 0.04 #Drizzled frame size
fixcenter = False
run_MCMC = False
zp= 28.

psf, QSO_img = pyfits.getdata('sim_ID_0/Drz_PSF.fits'),  pyfits.getdata('sim_ID_0/Drz_QSO_image.fits')

fm_size = len(QSO_img)
stdd = np.std(QSO_img[int(fm_size/3):int(fm_size*2/3),5:int(fm_size/3)]) #estimate background noise from empty region.

#framesize = 61
#ct = int((len(QSO_img) - framesize)/2)
#QSO_img = QSO_img[ct:-ct,ct:-ct]
ID= 0
folder = "sim_ID_{0}".format(ID)
f = open(folder+"/sim_info.txt","r")
string = f.read()
lines = string.split('\n')   # Split in to \n
exp_info = [lines[i] for i in range(len(lines)) if 'exposure' in lines[i]][0]
exp = float(re.findall(r'\d+\.\d+', exp_info)[0])

exptime = exp * 8  #Total exposure is the sum of the eight dithered image.
QSO_std = (abs(QSO_img/exptime)+stdd**2)**0.5

In [ ]:
# input the objects components and parameteres
#objs, Q_index = detect_obj(QSO_img,pltshow = pltshow)
#qso_info = objs[Q_index]
#obj = [objs[i] for i in range(len(objs)) if i != Q_index]
fixed_source = []
kwargs_source_init = []
kwargs_source_sigma = []
kwargs_lower_source = []
kwargs_upper_source = []
fixed_source.append({})  
kwargs_source_init.append({'R_sersic': 0.3, 'n_sersic': 2., 'e1': 0., 'e2': 0., 'center_x': 0., 'center_y': 0.})
kwargs_source_sigma.append({'n_sersic': 0.5, 'R_sersic': 0.5, 'e1': 0.1, 'e2': 0.1, 'center_x': 0.1, 'center_y': 0.1})
kwargs_lower_source.append({'e1': -0.5, 'e2': -0.5, 'R_sersic': 0.1, 'n_sersic': 0.3, 'center_x': -0.5, 'center_y': -0.5})
kwargs_upper_source.append({'e1': 0.5, 'e2': 0.5, 'R_sersic': 3., 'n_sersic': 7., 'center_x': 0.5, 'center_y': 0.5})

source_params = [kwargs_source_init, kwargs_source_sigma, fixed_source, kwargs_lower_source, kwargs_upper_source]

tag = None
#tag = folder+'example' #If need to save the inferred figures
source_result, ps_result, image_ps, image_host, error_map=fit_qso(QSO_img, psf_ave=psf, psf_std = None,
                                                                  source_params=source_params, fixcenter=fixcenter,
                                                                  pix_sz = pix_scale, no_MCMC = (run_MCMC==False),
                                                                  QSO_std =QSO_std, tag=tag, deep_seed= deep_seed, pltshow=pltshow,
                                                                  corner_plot=False, flux_ratio_plot=True, dump_result=run_MCMC, pso_diag =True)


plot_compare=True
fits_plot =True

result = transfer_to_result(data=QSO_img, pix_sz = pix_scale,
                            source_result=source_result, ps_result=ps_result, image_ps=image_ps, image_host=image_host, error_map=error_map,
                            zp=zp, fixcenter=fixcenter,ID='Example', tag=tag, plot_compare = plot_compare)


Computing the PSO ...
10
20
30
40
50
60
70
80
90


In [ ]:
print("The truth:")
with open("sim_ID_0/sim_info.txt") as f: # The with keyword automatically closes the file when you are done
    print(f.read())
                     
print("The inferred results:")                     
print("host_flux:",  result['host_amp'])
print("host Reff:",  result['R_sersic'])
print("host n:",  result['n_sersic'])
print("host q:",  result['q'])
print("AGN flux :",  result['QSO_amp'])